# **Tarjeta de datos 3**

### Hipótesis 3: "A mayor número de personas en un hogar es más probable que se invierta en energía renovable"

- Por un lado, se utiliza *tipo_nucleo_familiar.csv* para procesarlo y hacer una mejor agrupación de hogares en España. 
- Por otro lado, mediante *dispositivos_renovable.csv* se obtiene el porcentaje de dispositivos que son renovables en cada provincia.

- Con estos datos, se podrá estudiar la correlación que hay entre los diferentes núcleos famliares y el porcentaje de dispositivos renovables. Para ello, en primer lugar, habrá que realizar un mejor procesamiento de los datos, limpiando y redimensionando los datasets. Así, se conseguirá tener nuestro dataset gold para esta tarjeta de datos.

## **Carga de datos**

In [108]:
# Importar librerías
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import qqplot

SILVER_DATA_PATH = os.path.join("..", "data/silver/")
GOLD_DATA_PATH = os.path.join("..", "data/gold/")

In [109]:
# Dispositivos de energía renovable por provincia e ingresos.
dispositivos_renovable_df = pd.read_csv(SILVER_DATA_PATH+"dispositivos_renovable.csv", sep = ";", encoding = "latin")

#dataframe info.
dispositivos_renovable_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 4 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   Provincias                                   470 non-null    object
 1   Ingresos netos                               470 non-null    object
 2   Dispone de dispositivo de energía renovable  470 non-null    object
 3   Total                                        470 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 14.8+ KB


In [110]:
# Cargar el DataFrame con data type str para evitar errores de lectura, luego se cambiará el tipo de dato si es necesario
tipo_nucleo_familiar_df = pd.read_csv(SILVER_DATA_PATH + "tipo_nucleo_familiar.csv", sep=";", encoding="latin")

# Ver la información del DataFrame
tipo_nucleo_familiar_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Provincias               1128 non-null   object
 1   Tipo de núcleo familiar  1128 non-null   object
 2   Número de hijos          1128 non-null   object
 3   Total                    1128 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 35.4+ KB


## **Procesamiento**

### tipo_nucleo_familiar.csv


- Se ve que en el dataset tipo_nucleo_familar_df tiene valores iguales a cero en la columna 'Total'. Se comprueba que filas son y se eliminarán si no afectan a la composición y calidad de los datos.

In [111]:
tipo_nucleo_familiar_df.head(13)

,Provincias,Tipo de núcleo familiar,Número de hijos,Total
0,Albacete,"Pareja casada, con o sin hijos convivientes",0 hijos conviviendo,27344
1,Albacete,"Pareja casada, con o sin hijos convivientes",1 hijo conviviendo,21719
2,Albacete,"Pareja casada, con o sin hijos convivientes",2 o más hijos conviviendo,31743
3,Albacete,"Pareja casada, con o sin hijos convivientes",0 hijos conviviendo menores de 25 años,39498
4,Albacete,"Pareja casada, con o sin hijos convivientes",1 hijo conviviendo menor de 25 años,16930
5,Albacete,"Pareja casada, con o sin hijos convivientes",2 hijos o más conviviendo menores de 25 años,24377
6,Albacete,"Pareja no casada, con o sin hijos convivientes",0 hijos conviviendo,5669
7,Albacete,"Pareja no casada, con o sin hijos convivientes",1 hijo conviviendo,3085
8,Albacete,"Pareja no casada, con o sin hijos convivientes",2 o más hijos conviviendo,3555
9,Albacete,"Pareja no casada, con o sin hijos convivientes",0 hijos conviviendo menores de 25 años,5669


- Se eliminan aquellas filas que no aportan ningún valor. En este caso, aquellas que toman valor cero en *Total*

In [112]:
# Filas con valores 0 en la columna 'Total'
total_cero_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Total'] == 0]
print("Dimensión del dataframe con valores 0:", total_cero_df.shape)

print('\n',total_cero_df.head())

# Valores únicos de las columnas 'Tipo de núcleo familiar' y 'Número de hijos' para total_cero_df
valores_unicos = total_cero_df[['Tipo de núcleo familiar', 'Número de hijos']].drop_duplicates().values.tolist()
print('\n',valores_unicos)

Dimensión del dataframe con valores 0: (94, 4)

           Provincias            Tipo de núcleo familiar      Número de hijos  \
12          Albacete  Padre sólo con hijos convivientes  0 hijos conviviendo   
18          Albacete  Madre sóla con hijos convivientes  0 hijos conviviendo   
36  Alicante/Alacant  Padre sólo con hijos convivientes  0 hijos conviviendo   
42  Alicante/Alacant  Madre sóla con hijos convivientes  0 hijos conviviendo   
60           Almería  Padre sólo con hijos convivientes  0 hijos conviviendo   

    Total  
12      0  
18      0  
36      0  
42      0  
60      0  

 [['Padre sólo con hijos convivientes', '0 hijos conviviendo'], ['Madre sóla con hijos convivientes', '0 hijos conviviendo']]


- Se ve que en el dataframe las filas donde 'Total' es cero son aquellas que tienen la combinación: [['Padre sólo con hijos convivientes', '0 hijos conviviendo'], ['Madre sóla con hijos convivientes', '0 hijos conviviendo']]
    - Se eliminan esas filas.

In [113]:
#Eliminar filas donde 'Total' sea 0
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df[tipo_nucleo_familiar_df['Total'] != 0]

- Ahora, se hace una categorización más clara y concisa en *Tipo de núcleo familiar*:
    - Pareja casada.
    - Pareja no casada.
    - Familia monoparental

- Igualmente, se hará con *Número de hijos*. Se categorizará únicamente por número de hijos sin distinguir si son o no mayores de 25 años:
    - 0 hijos
    - 1 hijo
    - 2 o más hijos

In [114]:
# Reemplazar valores en la columna "Tipo de núcleo familiar"
tipo_nucleo_familiar_df['Tipo de núcleo familiar'] = tipo_nucleo_familiar_df['Tipo de núcleo familiar'].replace({
    'Pareja casada, con o sin hijos convivientes': 'Pareja casada',
    'Pareja no casada, con o sin hijos convivientes': 'Pareja no casada',
    'Padre sólo con hijos convivientes': 'Familia monoparental',
    'Madre sóla con hijos convivientes': 'Familia monoparental'
})

tipo_nucleo_familiar_df['Número de hijos'] = tipo_nucleo_familiar_df['Número de hijos'].replace({
    '0 hijos conviviendo': '0 hijos', '0 hijos conviviendo menores de 25 años': '0 hijos',
    '1 hijo conviviendo': '1 hijo', '1 hijo conviviendo menor de 25 años': '1 hijo',
    '2 o más hijos conviviendo': '2 hijos o más', '2 hijos o más conviviendo menores de 25 años': '2 hijos o más'
})
# Verificar los cambios
print(tipo_nucleo_familiar_df['Tipo de núcleo familiar'].unique())
print(tipo_nucleo_familiar_df['Número de hijos'].unique())

['Pareja casada' 'Pareja no casada' 'Familia monoparental']
['0 hijos' '1 hijo' '2 hijos o más']


- Agrupamos columnas con *Provincia*s, *Tipo de núcelo familiar* y *Número de hijos* iguales.

In [115]:
#reset_index() es utilizado para convertir la Serie generada por groupby en un dataframe.
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df.groupby(['Provincias', 'Tipo de núcleo familiar', 'Número de hijos'])['Total'].sum().reset_index()

In [116]:
tipo_nucleo_familiar_df.head(18)

,Provincias,Tipo de núcleo familiar,Número de hijos,Total
0,Albacete,Familia monoparental,0 hijos,7262
1,Albacete,Familia monoparental,1 hijo,15905
2,Albacete,Familia monoparental,2 hijos o más,9786
3,Albacete,Pareja casada,0 hijos,66842
4,Albacete,Pareja casada,1 hijo,38649
5,Albacete,Pareja casada,2 hijos o más,56120
6,Albacete,Pareja no casada,0 hijos,11338
7,Albacete,Pareja no casada,1 hijo,6247
8,Albacete,Pareja no casada,2 hijos o más,7033
9,Alicante/Alacant,Familia monoparental,0 hijos,46286


- Para terminar con el procesamiento de *tipo_nucleo_familiar_df* se va a redimensionar el dataframe. Pues lo que se pretende es tener sólamente una fila por provincia. Entonces, se harán diferentes columnas para las combinaciones de valores de *Tipo de núcleo familiar* y *Número de hijos*.
    - Además, dimensionando por columnas, tendremos una mayor comprensión de los valores que toma el dataframe.

In [117]:
# Crear una nueva columna combinando 'Tipo de núcleo familiar' y 'Número de hijos'
tipo_nucleo_familiar_df['Categoria'] = tipo_nucleo_familiar_df['Tipo de núcleo familiar'] + ' con ' + tipo_nucleo_familiar_df['Número de hijos']

# Usar pivot para reorganizar la tabla sin incluir la categoría en el índice
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df.pivot(index='Provincias', columns='Categoria', values='Total')

# Para tener un DataFrame "normal", reiniciamos el índice
tipo_nucleo_familiar_df.reset_index(inplace=True)

# Eliminamos 'Categoría' como nombre de las columnas
tipo_nucleo_familiar_df.columns.name = None

# Mostrar el resultado
tipo_nucleo_familiar_df.head()

,Provincias,Familia monoparental con 0 hijos,Familia monoparental con 1 hijo,Familia monoparental con 2 hijos o más,Pareja casada con 0 hijos,Pareja casada con 1 hijo,Pareja casada con 2 hijos o más,Pareja no casada con 0 hijos,Pareja no casada con 1 hijo,Pareja no casada con 2 hijos o más
0,Albacete,7262,15905,9786,66842,38649,56120,11338,6247,7033
1,Alicante/Alacant,46286,106182,50846,371095,180545,233971,73675,42475,22805
2,Almería,12367,36792,17348,118470,65774,103918,24407,16456,12945
3,Araba/Álava,6453,18609,8642,62806,30008,38515,13852,7208,4968
4,Asturias,34988,70807,21841,224656,115520,81680,40912,14393,9763


### dispositivos_renovable_df

In [118]:
dispositivos_renovable_df.head(20)

,Provincias,Ingresos netos,Dispone de dispositivo de energía renovable,Total
0,Albacete,3.000 euros o más,No,13390
1,Albacete,3.000 euros o más,Si,1344
2,Albacete,De 1.000 euros a menos de 1.500 euros,No,38883
3,Albacete,De 1.000 euros a menos de 1.500 euros,Si,2260
4,Albacete,De 1.500 euros a menos de 2.000 euros,No,22329
5,Albacete,De 1.500 euros a menos de 2.000 euros,Si,956
6,Albacete,De 2.000 euros a menos de 3.000 euros,No,24122
7,Albacete,De 2.000 euros a menos de 3.000 euros,Si,1080
8,Albacete,Menos de 1.000 euros,No,47531
9,Albacete,Menos de 1.000 euros,Si,954


- Agrupar por provincia y sumar las viviendas con y sin dispositivos

In [119]:
# Agrupar por provincia (groupby) y sumar las viviendas con y sin dispositivos
dispositivos_renovable_df = dispositivos_renovable_df.groupby(['Provincias', 'Dispone de dispositivo de energía renovable'])['Total'].sum().unstack()

In [120]:
dispositivos_renovable_df.head()

Dispone de dispositivo de energía renovable,No,Si
Provincias,,
Albacete,146255,6594
Alicante/Alacant,735245,30923
Almería,248102,17856
Araba/Álava,126383,14741
Asturias,431584,22865


- Crear dataframe con los resultados obtenidos:
- dispositivos_porcentaje_df:
    - provincias: nombrede la provincia.
    - porcentaje_con_dispositivo: porcentaje de viviendas con dispositivos renovable (sobre 100).

In [121]:
# Dataframe final con los resultados relevantes
dispositivos_renovable_df = dispositivos_renovable_df[['Si']].reset_index()

dispositivos_renovable_df.columns.name = None
dispositivos_renovable_df.rename(columns={'Si': 'Viviendas con dispositivos de energía renovable'}, inplace=True)

dispositivos_renovable_df.head()

,Provincias,Viviendas con dispositivos de energía renovable
0,Albacete,6594
1,Alicante/Alacant,30923
2,Almería,17856
3,Araba/Álava,14741
4,Asturias,22865


## Integrar datos

- Hacer merge de datos de los datos de porcentaje de dispositivos renovable y tipos de núcleo familiares en un mismo dataframe.

- Se combinan los datos de las columnas:
    - Provincias (dispositivos_porcentaje_df y media_hogar_df)
    - Porcentaje con dispositivos (dispositivos_porcentaje_df)
    - Media hogar (media_hogar_df)

- Antes de juntar ambos datasets, se comprueba que contienen las mismas provincias.

In [122]:
# Listar las provincias únicas de cada DataFrame
provincias_dispositivos = set(dispositivos_renovable_df['Provincias'].unique())
provincias_hogar = set(tipo_nucleo_familiar_df['Provincias'].unique())

# Provincias presentes en un DataFrame pero no en el otro
provincias_solo_dispositivos = provincias_dispositivos - provincias_hogar
provincias_solo_hogar = provincias_hogar - provincias_dispositivos

# Mostrar los resultados
print("Provincias en dispositivos_porcentaje_df pero no en media_hogar_df:")
print(provincias_solo_dispositivos)

print("\nProvincias en media_hogar_df pero no en dispositivos_porcentaje_df:")
print(provincias_solo_hogar)

# Comprobar si tienen valores exactamente iguales
valores_iguales = provincias_dispositivos == provincias_hogar
print("\n¿Ambos DataFrames tienen exactamente las mismas provincias?:", valores_iguales)


Provincias en dispositivos_porcentaje_df pero no en media_hogar_df:
set()

Provincias en media_hogar_df pero no en dispositivos_porcentaje_df:
set()

¿Ambos DataFrames tienen exactamente las mismas provincias?: True


- Ambas dataframes contienen las mismas provincias, se continúa con el merge. Los provincias tienne el mismo orden en ambos dataframes. Por lo tanto, concatenándolas y eliminándo una columna Provincia, ya no se tendrá columnas replicadas.

In [123]:
# Eliminar la columna 'Provincias' de tipo_nucleo_familiar_df pues ya está en dispositivos_renovable_df
tipo_nucleo_familiar_df = tipo_nucleo_familiar_df.drop(columns=['Provincias'])

# Concatenar las DataFrames
tarjeta_3 = pd.concat([dispositivos_renovable_df, tipo_nucleo_familiar_df], axis=1)

- **Mostrar cómo queda el dataframe procesado para esta tarjeta de datos.**

In [124]:
tarjeta_3.head() #mostrar cómo queda el dataframe final

,Provincias,Viviendas con dispositivos de energía renovable,Familia monoparental con 0 hijos,Familia monoparental con 1 hijo,Familia monoparental con 2 hijos o más,Pareja casada con 0 hijos,Pareja casada con 1 hijo,Pareja casada con 2 hijos o más,Pareja no casada con 0 hijos,Pareja no casada con 1 hijo,Pareja no casada con 2 hijos o más
0,Albacete,6594,7262,15905,9786,66842,38649,56120,11338,6247,7033
1,Alicante/Alacant,30923,46286,106182,50846,371095,180545,233971,73675,42475,22805
2,Almería,17856,12367,36792,17348,118470,65774,103918,24407,16456,12945
3,Araba/Álava,14741,6453,18609,8642,62806,30008,38515,13852,7208,4968
4,Asturias,22865,34988,70807,21841,224656,115520,81680,40912,14393,9763


- **Se guardan los datos como csv en carpeta data/gold**:


In [125]:
with open(GOLD_DATA_PATH + "tarjeta_3.csv", "w") as f:
    tarjeta_3.to_csv(f, sep=';', encoding='utf-8')

## Profiling de la tarjeta para la hipótesis 3

- En este apartado, primeramente, se crea un Profile Report en formato html. Las conclusiones extraídas relevantes de este report se comentarán en el entregable 2.

In [126]:
from  ydata_profiling import ProfileReport

profile = ProfileReport(tarjeta_3, title="Tarjeta 3")
profile.to_file("tarjeta_3.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 76.92it/s]


- A continuación, se procede con un profiling de los datos realizados aparte.

In [127]:
tarjeta_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 11 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Provincias                                       47 non-null     object
 1   Viviendas con dispositivos de energía renovable  47 non-null     int64 
 2   Familia monoparental con 0 hijos                 47 non-null     int64 
 3   Familia monoparental con 1 hijo                  47 non-null     int64 
 4   Familia monoparental con 2 hijos o más           47 non-null     int64 
 5   Pareja casada con 0 hijos                        47 non-null     int64 
 6   Pareja casada con 1 hijo                         47 non-null     int64 
 7   Pareja casada con 2 hijos o más                  47 non-null     int64 
 8   Pareja no casada con 0 hijos                     47 non-null     int64 
 9   Pareja no casada con 1 hijo                  